# Import packages
run the following cell to import necessary packages to run the analysis

In [1]:
import os
from pathlib import Path

import dask.array as da
import xarray as xr
import tifffile
import ngff_zarr

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
# from pylibCZIrw import czi as pyczi
# from bioio import BioImage
# import bioio_czi

# Reading data

In [2]:
data_path = Path('/mnt/Data/nuno_martins/Image_data/Ex010_Re01/ExM/20250325/test_dir/')
filelist = os.listdir(data_path)
print(len(filelist))

2


In [3]:
filelist = [x for x in filelist if x.find('.czi')>0]
filelist.sort()
filelist

['NPM_Ex010_Re03_gel_Im-02_AcquisitionBlock3_pt3_sub1-Scene-1-Airyscan Processing-05_2.czi']

In [4]:
file_index = 0
file_path = str(data_path)+'/'+filelist[file_index]
filename = filelist[file_index][:filelist[file_index].index('.czi')]
print(file_path)
print(filename)

/mnt/Data/nuno_martins/Image_data/Ex010_Re01/ExM/20250325/test_dir/NPM_Ex010_Re03_gel_Im-02_AcquisitionBlock3_pt3_sub1-Scene-1-Airyscan Processing-05_2.czi
NPM_Ex010_Re03_gel_Im-02_AcquisitionBlock3_pt3_sub1-Scene-1-Airyscan Processing-05_2


In [5]:
save_path = Path(str(data_path)+'/split_czi')
save_path.mkdir(parents=True, exist_ok=True)
print(save_path)

/mnt/Data/nuno_martins/Image_data/Ex010_Re01/ExM/20250325/test_dir/split_czi


___
# Read raw data
reading raw tile data from `czi` tile without stitching for visualization.

In [6]:
from bioio import BioImage
import bioio_czi

In [9]:
img = BioImage(
    file_path, 
    reader=bioio_czi.Reader, 
    reconstruct_mosaic=False,
    include_subblock_metadata=True,
    use_aicspylibczi=True,
    )
img.dims, img.dims.order

(<Dimensions [M: 4, T: 1, C: 3, Z: 238, Y: 4084, X: 4084]>, 'MTCZYX')

In [10]:
img.scale, img.channel_names

(Scale(T=None, C=None, Z=0.4, Y=0.04708196466619325, X=0.04708196466619325),
 [np.str_('AF647-T1'), np.str_('AF488-T2'), np.str_('DAPI-T3')])

In [12]:
# img.scenes
for scene in img.scenes:
    img.set_scene(scene)
    print(img.dims, img.current_scene)

<Dimensions [M: 4, T: 1, C: 3, Z: 238, Y: 4084, X: 4084]> TR1


In [ ]:
# def getDimensionPosition(dimension_order, dimension_name):
#     return dimension_order.index(dimension_name)

In [14]:
from pylibCZIrw import czi as pyczi

In [16]:
for name in img.channel_names:
    print(name)

AF647-T1
AF488-T2
DAPI-T3


In [23]:
n_tiles = img.dims['M'][0]

img.scale
scale_x = img.scale.X
scale_y = img.scale.Y
scale_z = img.scale.Z

for tile in tqdm(range(n_tiles)):
    img_data = img.get_image_dask_data(img.dims.order[1:], M=tile) # TODO search for dim index
    img_data_tile = img_data.compute()

    ch_names  = {}
    for i in range(len(img.channel_names)):
        ch_names[i] = img.channel_names[i]
    

    tile_save_path = str(save_path)+'/'+filename+'_tile_'+str(tile+1).zfill(2)+'.czi'

    with pyczi.create_czi(tile_save_path, exist_ok=True) as czidoc_w:
        for t in range(img.dims['T'][0]):
            for c in range(img.dims['C'][0]):
                for z in range(img.dims['Z'][0]):
                    temp_image = img_data_tile[t, c, z, ...]
                    czidoc_w.write(
                        data=temp_image,
                        plane = {
                            'T': t,
                            'Z': z,
                            'C': c,
                        },
                        compression_options = "zstd0:ExplicitLevel=2",
                    )
        
        czidoc_w.write_metadata(
            filename+'_tile_'+str(tile+1).zfill(2),
            channel_names = ch_names,
            scale_x = scale_x,
            scale_y = scale_y,
            scale_z = scale_z,
        )



  0%|          | 0/4 [00:00<?, ?it/s]